In [1]:
import sys
import logging
import os.path as p
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna

from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data.dataloader import DataLoader
from definitions import *
from model_helper_functions import *
from dataset_helper_functions import *
from sent_nn import SentNN
from sentence_transformers import SentenceTransformer
from debates_dataset import DebatesDataset
from optuna.trial import TrialState
from torchvision import transforms
# my transforms
from transforms import *

In [2]:
data = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_uw_ratio = 0
dataset_frac = 0.1
worthy_frac = 0.2
rs = 22

Function for loading data.

In [3]:
def load_data():
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
        ],
        'val': [
            p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
        ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_csv(dpaths[0], sep='\t', index_col=False)

        except Exception as e:
            print(e.args)
            exit()

Datasets and DataLoaders, takes trial as input to be able to suggest values for variables.

In [4]:
def get_loaders(trial):
    global train_uw_ratio
    # dev_df, test_df, train_df, val_df = data.values()
    subsets = {}
    for k, df in data.items():

        n_subset = int(len(df)*dataset_frac)

        worthy_df = df.loc[df['label'] == 1]
        n_worthy = int(min(n_subset*worthy_frac, len(worthy_df)))
        worthy_df = worthy_df.sample(n=n_worthy, random_state=rs)

        unworthy_df = df.loc[df['label'] == 0].sample(
            n=n_subset-n_worthy,
            random_state=rs
        )
        if k == 'train':
            train_uw_ratio = len(unworthy_df) / len(worthy_df)
        # sample(frac=1.0) -> shuffle
        subsets[k] = worthy_df.append(unworthy_df).sample(frac=1.0, random_state=rs, ignore_index=True)

    transform_pipeline = None

    train_dd = DebatesDataset(data=subsets['train'], transform=transform_pipeline)
    val_dd = DebatesDataset(data=subsets['val'], transform=transform_pipeline)
    test_dd = DebatesDataset(data=subsets['test'], transform=transform_pipeline)

    # batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    batch_size = 32
    train_loader = DataLoader(train_dd, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dd, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_dd, batch_size=batch_size, shuffle=True, drop_last=True)

    return train_loader, val_loader, test_loader

Model setup + training loop

In [5]:
def objective(trial):
    # unused is test_loader
    train_loader, val_loader, _ = get_loaders(trial)

    # hyperparams opt
    # emb_model_name = trial.suggest_categorical(
    #     'embedding_model_name',
    #     ['all-mpnet-base-v2', 'all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1']
    # )
    emb_size_map = {
        'all-mpnet-base-v2': 768,
        'all-MiniLM-L6-v2': 384,
        'multi-qa-mpnet-base-dot-v1': 768
    }
    emb_model_name = 'all-MiniLM-L6-v2'
    embedding_model = SentenceTransformer(emb_model_name, device=device)
    
    # dropout = trial.suggest_float('dropout', 0.0, 0.5, step=0.01)
    # lstm_dropout = trial.suggest_float('lstm_dropout', 0.0, 0.3, step=0.05)
    # hidden_dim = trial.suggest_categorical('hidden_dim', [64, 128, 256])
    dropout = 0.4
    lstm_dropout = 0.0
    hidden_dim = 128
    model = SentNN(
        embeddings_dim=emb_size_map[emb_model_name],
        sentence_level_feature_dim=0,
        dropout=dropout
    ).to(device)    

    # lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    # opt_weight_decay = trial.suggest_float('optimizer_weigth_decay', 0, 0.1, log=True)
    lr = 0.001
    opt_weight_decay = 0
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=opt_weight_decay)

    # pos_weight = trial.suggest_categorical('pos_weight', [1.0, train_uw_ratio])
    pos_weight = train_uw_ratio
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))

    n_epochs = 15
    threshold = 0.5
    # training
    for epoch in range(n_epochs):
        losses, val_losses = [], []
        model.train()
        for ids, sentences, labels, features in train_loader:
            labels = labels.float().to(device)

            optimizer.zero_grad()

            embeddings = embedding_model.encode(sentences, convert_to_tensor=True)
            output = model(embeddings)
            loss = criterion(output, labels)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()

        model.eval()
        y_pred, y_true = [], []
        with torch.no_grad():
            for val_ids, val_sentences, val_labels, val_features in val_loader:
                val_labels = val_labels.float().to(device)

                val_embeddings = embedding_model.encode(val_sentences, convert_to_tensor=True)
                pred = model(val_embeddings)
                loss = criterion(pred, val_labels)
                val_losses.append(loss.item())

                pred = torch.sigmoid(pred)

                pred = (pred > threshold).int()
                y_pred.extend(pred.tolist())
                y_true.extend(val_labels.tolist())

        print(f'train_loss: {np.average(losses)} | val_loss: {np.average(val_losses)}')
        cr = classification_report(y_true, y_pred, output_dict=True, digits=6)
        recall_p = cr['1.0']['recall']
        print(recall_p)
        # trial.report(recall_p, epoch)

        # # Handle pruning based on the intermediate value.
        # if trial.should_prune():
        #     raise optuna.exceptions.TrialPruned()

    "Done."
    return recall_p

In [6]:
load_data()
print('final recall: ', objective(None))

train_loss: 1.0980991460382938 | val_loss: 1.0810778190692265
0.6266666666666667
train_loss: 1.0821352787315845 | val_loss: 1.068364456295967
0.7105263157894737
train_loss: 1.0630025292436283 | val_loss: 1.0453306833902996
0.6933333333333334
train_loss: 1.0430305500825245 | val_loss: 1.0414480914672215
0.7402597402597403
train_loss: 1.0274585969746113 | val_loss: 1.0269260555505753
0.7532467532467533
train_loss: 1.0090080238878727 | val_loss: 1.0133405576149623
0.7532467532467533
train_loss: 1.0055760083099206 | val_loss: 0.9973104347785314
0.75
train_loss: 0.9828560600678126 | val_loss: 0.9834993282953898
0.7763157894736842
train_loss: 0.9697593164940675 | val_loss: 0.9715106139580408
0.7631578947368421
train_loss: 0.962579257786274 | val_loss: 0.9596485793590546
0.7466666666666667
train_loss: 0.9516235080858072 | val_loss: 0.9530285050471624
0.75
train_loss: 0.9392038707931837 | val_loss: 0.9482677976290385
0.7532467532467533
train_loss: 0.9312100298702717 | val_loss: 0.9415737887223

In [ ]:
# # optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
# study = optuna.create_study(
#     study_name=f'sent_nn_df{dataset_frac}_wf{worthy_frac}',
#     sampler=optuna.samplers.RandomSampler(rs),
#     pruner=optuna.pruners.MedianPruner(),
#     direction='maximize'
# )
# study.optimize(objective, n_trials=2)

# torch.save(study, os.path.join(EXP_DIR_PATH, 'sent-nn', 'optimization', f'{study.study_name}.pkl'))